In [4]:
!pip install numpy
!pip install transformers
!pip install peft
!pip install sklearn
!pip install pandas
!pip install evaluate
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00


## Import Packages

In [5]:
import pandas as pd

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

## Load Dataset

In [7]:
df = pd.read_csv('/content/data/training_data_elements_of_statistical_learning.csv')

In [8]:
df.head()

,Question,Answer
0,What is the main purpose of cross-validation?,Cross-validation is used to assess how well a ...
1,How does Lasso Regression differ from Ridge Re...,"Lasso regression applies an L1 penalty, encour..."
2,What is regularization in machine learning?,Regularization is a technique to reduce model ...
3,Explain the bias-variance tradeoff.,The bias-variance tradeoff is the balance betw...
4,What is the purpose of Random Forests?,Random Forests reduce variance by averaging mu...


### Tokenize the data

In [38]:
from transformers import GPT2Tokenizer

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenized_examples=[]

for index, row in df.iterrows():
  qa_pair = f"Question: {row['Question']} Answer: {row['Answer']}"

  # Tokenize the text
  tokenized = tokenizer(
      qa_pair,
      max_length=512, # This can be adjusted
      truncation=True,
      return_tensors='pt' # Returns PyTorch tensors for use in training
  )

  # Append tokenized data, where labels are identical to input_ids
  tokenized_examples.append({
      'input_ids': tokenized['input_ids'].squeeze(),
      'attention_mask': tokenized['attention_mask'].squeeze(),
      'labels': tokenized['input_ids'].squeeze()  # The labels here are the expected outputs
  })
tokenized_dataset = tokenized_examples

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # We need to access each tokenized entry
        item = self.data[idx]
        return {
            'input_ids': item['input_ids'],
            'attention_mask': item['attention_mask'],
            'labels': item['labels']
        }

# Convert the preprocessed list of examples to a PyTorch Dataset
qa_dataset = QADataset(tokenized_dataset)



## Setup Model

In [37]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoModelForCausalLM, AutoModelForQuestionAnswering

# Load the GPT-2 model
model = AutoModelForCausalLM.from_pretrained('gpt2')
qa_model = AutoModelForQuestionAnswering.from_pretrained('gpt2')


from peft import get_peft_model, LoraConfig, TaskType

# LoRA configuration for GPT-2 fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Causal Language Model
    inference_mode=False,
    r=16,  # Rank of the LoRA matrices (determines complexity of the adaptation)
    lora_alpha=32,  # Scaling factor to control LoRA update magnitude
    lora_dropout=0.1  # Dropout rate during fine-tuning
)

# Wrap GPT-2 model with LoRA for parameter-efficient fine-tuning
model = get_peft_model(model, lora_config)



Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [34]:
from transformers import TrainingArguments, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory where the model checkpoints will be saved
    num_train_epochs=3,  # Number of training epochs (you can adjust this)
    per_device_train_batch_size=4,  # Batch size per GPU (you can tweak this to fit in GPU memory)
    learning_rate=5e-5,  # Learning rate
    weight_decay=0.01,  # Weight decay to prevent overfitting
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # How often to log progress
    save_steps=100,  # Save model every 100 steps
    save_total_limit=2,  # Keep only the last 2 checkpoints
    evaluation_strategy='steps',
    eval_steps=100,
    load_best_model_at_end=True  # Load the best model after training
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Train the Model

In [ ]:
# Make sure input_ids and labels are the same (labels are required for language modeling tasks)
def preprocess_function(examples):
    return {'input_ids': examples['input_ids'], 'labels': examples['input_ids']}

# Assuming `qa_dataset` is your tokenized dataset, you can map it as follows:
train_dataset = qa_dataset.map(preprocess_function)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer  # The tokenizer we previously loaded
)

# Start training
trainer.train()
